### Churn Prediction Project (Classification)

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [68]:
import urllib.request

url = "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv"
file_path = "data/Telco-Customer-Churn.csv"

#urllib.request.urlretrieve(url, file_path)

In [69]:
df = pd.read_csv('data/Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [70]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [71]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [72]:
df.totalcharges

0         29.85
1        1889.5
2        108.15
3       1840.75
4        151.65
         ...   
7038     1990.5
7039     7362.9
7040     346.45
7041      306.6
7042     6844.5
Name: totalcharges, Length: 7043, dtype: object

In [73]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

In [74]:
df.churn.head()

0     no
1     no
2    yes
3     no
4    yes
Name: churn, dtype: object

In [75]:
df.churn = (df.churn == 'yes').astype(int)

### Setting up the validation framework

In [76]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [77]:
len(df_full_train), len(df_test)

(5634, 1409)

In [78]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [79]:
len(df_train), len(df_val)

(4225, 1409)

In [80]:
y_train, y_val, y_test = None, None, None

for i in ['train', 'val', 'test']:
   df = globals()[f'df_{i}']
   #print(f'y_{i} = {globals().get(f"y_{i}", "not defined")}')

   y = globals()[f'y_{i}']
   
   df = df.reset_index(drop=True)
   
   y = df.churn.values

   df = df.drop(columns=['churn'])

   # Reassign the variables to the updated versions
   globals()[f'df_{i}'] = df
   globals()[f'y_{i}'] = y

In [81]:
df_train.totalcharges

0         64.55
1       5656.75
2       1218.25
3         20.20
4       3778.85
         ...   
4220     526.95
4221    4039.00
4222     603.00
4223     903.70
4224    1801.10
Name: totalcharges, Length: 4225, dtype: float64

### EDA

* Check missing values
* Look at the target variable (churn)
* Look at numerical and categorical variables

In [82]:
df_full_train.isnull().sum() 

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [83]:
df_full_train.reset_index(drop=True, inplace=True)

In [84]:
df_full_train.churn.value_counts(normalize=True)

churn
0    0.734469
1    0.265531
Name: proportion, dtype: float64

In [85]:
df_full_train.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges        float64
churn                 int32
dtype: object

### Feature importance: Churn rate and risk ratio

In [86]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [87]:
categorical = categorical_columns.copy()
categorical.remove('customerid')
categorical.remove('churn')
categorical.remove('totalcharges')
categorical.append('seniorcitizen')

In [88]:
df_full_train[categorical].nunique()

gender              2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
seniorcitizen       2
dtype: int64

### Churn Rate

In [89]:
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

0.26047800484932454

In [90]:
global_churn = df_full_train.churn.mean()
global_churn

0.2655307064252751

In [91]:
df_full_train.partner.value_counts()

partner
no     2904
yes    2730
Name: count, dtype: int64

In [92]:
churn_partner = df_full_train[df_full_train.partner == 'yes']['churn'].mean()
churn_partner

0.20073260073260074

In [93]:
churn_no_partner = df_full_train[df_full_train.partner == 'no']['churn'].mean()
churn_no_partner

0.32644628099173556

In [94]:
global_churn - churn_partner, global_churn - churn_no_partner

(0.06479810569267436, -0.06091557456646046)

### Risk Ratio

In [95]:
churn_partner/global_churn

0.7559675618499149

In [96]:
churn_no_partner/global_churn

1.2294106598311754

In [104]:
for category in categorical:
    print(category)
    df_group = df_full_train.groupby(category)['churn'].agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)
    print("\n\n")

gender


,mean,diff,risk
gender,,,
female,0.270841,0.005310,1.019998
male,0.260478,-0.005053,0.980971





partner


,mean,diff,risk
partner,,,
no,0.326446,0.060916,1.229411
yes,0.200733,-0.064798,0.755968





dependents


...

,mean,diff,risk
paperlessbilling,,,
no,0.16414,-0.101390,0.618159
yes,0.33594,0.070409,1.265164





paymentmethod


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.174475,-0.091056,0.657080
credit_card_(automatic),0.152404,-0.113126,0.573961
electronic_check,0.449921,0.184390,1.694420
mailed_check,0.190328,-0.075203,0.716782





seniorcitizen


,mean,diff,risk
seniorcitizen,,,
0,0.237098,-0.028433,0.892922
1,0.413907,0.148377,1.558793
